In [11]:
import requests
import pandas as pd
import json
import numpy as np

### url格式

### 主要調整
- count 該頁面顯示的商品數量
- keyword 查詢商品名稱
- offset 第二頁的起始頁碼(依據前一頁查詢的商品數量決定)

In [ ]:
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'

### 查詢

### count : 固定為 96
### 查詢商品 : cheese
### oddset : 第一頁為0

In [16]:
kw = 'magazine'
number = 0
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
data =  requests.get(url.format(kw=kw,number=number)).json()

### 資料完整樣式

In [17]:
data

{'search_recommendations': {'query': 'magazine'},
 'search_response': {'items': {'Item': [{'subscribable': False,
     'package_dimensions': {'weight': '1.04',
      'weight_unit_of_measure': 'POUND',
      'width': '13.04',
      'depth': '13.91',
      'height': '6.8',
      'dimension_unit_of_measure': 'INCH'},
     'title': 'Mesh Magazine File Black - Made By Design&#8482;',
     'tcin': '53820984',
     'type': 'Stand Alone',
     'dpci': '081-08-0268',
     'upc': '490810802688',
     'url': '/p/mesh-magazine-file-black-made-by-design-8482/-/A-53820984',
     'description': 'A functional and stylish piece to display your magazines, journals, periodicals and other publications, the Mesh Magazine File Holder from Made by Design™ keeps your desk in tip-top shape. Constructed from durable mesh steel, this magazine file holder brings organization to your office desk, while the classic black finish brings a contemporary update to your space. Boasting a vertical profile, this magazine h

In [10]:
commodity_url = ['https://www.target.com'+i['url'] for i in data['search_response']['items']['Item']]
commodity_url

['https://www.target.com/p/shredded-mexican-style-cheese-8oz-good-38-gather-8482/-/A-54337113',
 'https://www.target.com/p/shredded-mozzarella-cheese-8oz-good-gather-8482/-/A-54337119',
 'https://www.target.com/p/colby-jack-cheese-8oz-good-gather-8482/-/A-54340528',
 'https://www.target.com/p/finely-shredded-cheddar-jack-cheese-8oz-good-gather-8482/-/A-54337115',
 'https://www.target.com/p/mild-cheddar-deli-sliced-cheese-8oz-12-slices-good-38-gather-8482/-/A-54340568',
 'https://www.target.com/p/colby-jack-deli-sliced-cheese-8oz-12-slices-good-38-gather-8482/-/A-54340560',
 'https://www.target.com/p/kraft-singles-american-cheese-slices-16ct/-/A-12955140',
 'https://www.target.com/p/colby-jack-cheese-snack-bars-9oz-12ct-good-38-gather-8482/-/A-54349793',
 'https://www.target.com/p/sharp-cheddar-cheese-8oz-good-gather-8482/-/A-54340519',
 'https://www.target.com/p/muenster-deli-sliced-cheese-8oz-12-slices-good-38-gather-8482/-/A-54340564',
 'https://www.target.com/p/shredded-sharp-chedda

### 商品名稱 'title'

### 商品圖片網址 是由images中的'base_url'及 'content_labels'中的'image_url'組成

### 商品價錢 'formatted_current_price'

### 商品文宣 'description'

### 商品評價 'average_rating'

### 商品評價分布 'rating_distribution' 會分別顯示'RatingValue'1-5的次數

### 商品3項指標得分數 'secondary_ratings_averages'包含 'Value', 'Quality', 'Taste'。分數是取裡面的'AverageRating'。 有些不是食物的商品不會有此指標

### 商品瀏覽數 'total_reviews'

### 商品評論 'top_reviews'，裡面又包含'user_nickname','title','rating','review_text'

### 商品描述 'bullet_description' 包含 'Contains','Form','State of Readiness','Package Quantity','Package type','Net weight'

### 商品特色(店家描述) 'soft_bullets'中的'bullets'

### 為了取得商品中的label info ， 先透過json取得商品所在網址，再透過商品_url = 'https://www.target.com'+'url網址' 進入商品頁面，再用request的方式取的label info 的資訊。不是所有商品都有這個

### 爬取一頁

In [ ]:
#商品名稱
commodity_name = [i['title'] for i in data['search_response']['items']['Item']]

#商品圖片網址
commodity_images = [i['images'][0]['base_url']+i['images'][0]['primary'] for i in data['search_response']['items']['Item']]

#商品價錢

commodity_price = [i['price']['formatted_current_price'] for i in data['search_response']['items']['Item']]

#商品url
commodity_url = ['https://www.target.com'+i['url'] for i in data['search_response']['items']['Item']]

#商品文宣
commodity_description = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_description.append(i['description'])
    except KeyError:
        commodity_description.append(np.nan)

#商品評價
commodity_average_rating = [i['average_rating'] for i in data['search_response']['items']['Item']]


#商品評價(次數)分布
#1星
commodity_RatingValue1 = [i['rating_distribution'][0]['Count'] for i in data['search_response']['items']['Item']]

#2星
commodity_RatingValue2 = [i['rating_distribution'][1]['Count'] for i in data['search_response']['items']['Item']]

#3星
commodity_RatingValue3 = [i['rating_distribution'][2]['Count'] for i in data['search_response']['items']['Item']]

#4星
commodity_RatingValue4 = [i['rating_distribution'][3]['Count'] for i in data['search_response']['items']['Item']]

#5星
commodity_RatingValue5 = [i['rating_distribution'][4]['Count'] for i in data['search_response']['items']['Item']]

#商品3項指標得分數  有些商品會沒有這些資料，應該移除
# Value
commodity_Value_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Value_score.append(i['secondary_ratings_averages']['Value']['AverageRating'])
    except KeyError:
        commodity_Value_score.append(np.nan)

#Quality
commodity_Quality_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Quality_score.append(i['secondary_ratings_averages']['Quality']['AverageRating'])
    except KeyError:
        commodity_Quality_score.append(np.nan)

#Taste
commodity_Taste_score = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_Taste_score.append(i['secondary_ratings_averages']['Taste']['AverageRating'])
    except KeyError:
        commodity_Taste_score.append(np.nan)

#商品瀏覽數
commodity_total_reviews = [i['total_reviews'] for i in data['search_response']['items']['Item']]

#商品特色(店家描述)
commodity_highlights = []
for i in data['search_response']['items']['Item']:
    try:
        commodity_highlights.append(i['soft_bullets']['bullets'])
    except KeyError:
        commodity_highlights.append(np.nan)

#商品描述 'bullet_description'
commodity_descriptio = [str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'","") for i in data['search_response']['items']['Item']]

### 裝資料容器

In [1]:
commodity_name = []
commodity_images = []
commodity_price = []
commodity_url = []
commodity_description = []
commodity_average_rating = []
commodity_RatingValue1 = []
commodity_RatingValue2 = []
commodity_RatingValue3 = []
commodity_RatingValue4 = []
commodity_RatingValue5 = []
commodity_Value_score = []
commodity_Quality_score = []
commodity_Taste_score = []
commodity_total_reviews = []
commodity_highlights = []
commodity_descriptio = []

### 爬取多頁

In [4]:
kw = 'cheese'
number = 0
while True:
    try:
        url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=96&keyword={kw}&offset={number}&pricing_store_id=3991&key=ff457966e64d5e877fdbad070f276d18ecec4a01'
        
        data =  requests.get(url.format(kw=kw,number=number)).json()
        
        
        for i in data['search_response']['items']['Item']:
            commodity_name.append(i['title']) #商品名稱
            commodity_images.append(i['images'][0]['base_url']+i['images'][0]['primary'])#商品圖片網址
            commodity_url.append('https://www.target.com'+i['url']) #商品url
            commodity_average_rating.append(i['average_rating']) #商品評價
            commodity_RatingValue1.append(i['rating_distribution'][0]['Count']) #商品評價(次數)分布 #1星
            commodity_RatingValue2.append(i['rating_distribution'][1]['Count']) #商品評價(次數)分布 #2星
            commodity_RatingValue3.append(i['rating_distribution'][2]['Count']) #商品評價(次數)分布 #3星
            commodity_RatingValue4.append(i['rating_distribution'][3]['Count']) #商品評價(次數)分布 #4星
            commodity_RatingValue5.append(i['rating_distribution'][4]['Count']) #商品評價(次數)分布 #5星
            commodity_total_reviews.append(i['total_reviews']) #商品瀏覽數
            commodity_descriptio.append(str(i['bullet_description']).replace('<B>','').replace('</B>','').replace("'",""))  #商品描述 'bullet_description'
       
        #商品價錢
        for i in data['search_response']['items']['Item']:
            try:
                commodity_price.append(i['price']['formatted_current_price'])
            except:
                commodity_price.append(np.nan)
        
        #商品文宣
        for i in data['search_response']['items']['Item']:
            try:
                commodity_description.append(i['description'])
            except KeyError:
                commodity_description.append(np.nan)
        
        #商品3項指標得分數
        # Value
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_Value_score.append(i['secondary_ratings_averages']['Value']['AverageRating'])
            except KeyError:
                commodity_Value_score.append(np.nan)
        
        #Quality
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_Quality_score.append(i['secondary_ratings_averages']['Quality']['AverageRating'])
            except KeyError:
                commodity_Quality_score.append(np.nan)
        
        #Taste
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_Taste_score.append(i['secondary_ratings_averages']['Taste']['AverageRating'])
            except KeyError:
                commodity_Taste_score.append(np.nan)
        
        #商品特色(店家描述)
        
        for i in data['search_response']['items']['Item']:
            try:
                commodity_highlights.append(i['soft_bullets']['bullets'])
            except KeyError:
                commodity_highlights.append(np.nan)

        number += 96
        #time.sleep(random.uniform(1, 5))
    except KeyError: #如果爬取的url 沒有上方的資料，就會產生KeyError，因此break
        break

### 變成DataFrame

In [5]:
df = pd.DataFrame({"commodity_name":commodity_name,'commodity_images':commodity_images,'commodity_price':commodity_price,'commodity_url':commodity_url,'commodity_description':commodity_description
,'commodity_average_rating':commodity_average_rating,'commodity_RatingValue1':commodity_RatingValue1,'commodity_RatingValue2':commodity_RatingValue2,
'commodity_RatingValue3':commodity_RatingValue3,'commodity_RatingValue4':commodity_RatingValue4,'commodity_RatingValue5':commodity_RatingValue1,                   
'commodity_Value_score':commodity_Value_score,'commodity_Quality_score':commodity_Quality_score,'commodity_Taste_score':commodity_Taste_score,                 
'commodity_total_reviews':commodity_total_reviews,'commodity_highlights':commodity_highlights,'commodity_descriptio':commodity_descriptio              
                 
                 
                 
                 
                 
                 
                  })


In [6]:
df.head()

,commodity_name,commodity_images,commodity_price,commodity_url,commodity_description,commodity_average_rating,commodity_RatingValue1,commodity_RatingValue2,commodity_RatingValue3,commodity_RatingValue4,commodity_RatingValue5,commodity_Value_score,commodity_Quality_score,commodity_Taste_score,commodity_total_reviews,commodity_highlights,commodity_descriptio
0,Shredded Mexican-Style Cheese - 8oz - Good &#3...,https://target.scene7.com/is/image/Target/GUES...,$2.29,https://www.target.com/p/shredded-mexican-styl...,Whether you're putting together a condiments b...,4.19,6,4,2,3,6,4.00,3.69,3.81,53,[8oz package of shredded Mexican-style cheese ...,"[Contains: Milk, Form: Shredded, State of Read..."
1,Shredded Mozzarella Cheese - 8oz - Good & Gath...,https://target.scene7.com/is/image/Target/GUES...,$2.29,https://www.target.com/p/shredded-mozzarella-c...,Maybe you're sprinkling cheese over a personal...,3.20,17,4,2,6,17,3.21,3.11,3.04,50,[8oz package of shredded low-moisture part-ski...,"[Contains: Milk, Form: Shredded, State of Read..."
2,Colby Jack Cheese - 8oz - Good & Gather&#8482;,https://target.scene7.com/is/image/Target/GUES...,$2.29,https://www.target.com/p/colby-jack-cheese-8oz...,For a versatile cheese that's as tasty on its ...,4.60,1,0,0,0,1,4.83,4.33,4.33,10,[Colby jack cheese block is a good source of c...,"[Contains: Milk, Form: Brick, State of Readine..."
3,Finely Shredded Cheddar Jack Cheese - 8oz - Go...,https://target.scene7.com/is/image/Target/GUES...,$2.29,https://www.target.com/p/finely-shredded-chedd...,"Give your signature chili, homemade broccoli-c...",3.83,1,0,1,1,1,2.50,2.50,2.00,6,[Shredded cheddar jack cheese is a good source...,"[Contains: Milk, Form: Shredded, State of Read..."
4,Mild Cheddar Deli Sliced Cheese - 8oz/12 slice...,https://target.scene7.com/is/image/Target/GUES...,$2.29,https://www.target.com/p/mild-cheddar-deli-sli...,Add a finishing touch to your sandwich with th...,3.92,3,3,2,2,3,3.67,3.00,4.00,25,"[8oz sliced cheese is perfect for sandwiches, ...","[Contains: Milk, Form: Sliced, State of Readin..."


In [6]:
df.isna().sum()

commodity_name                0
commodity_images              0
commodity_price               0
commodity_url                 0
commodity_description        43
commodity_average_rating      0
commodity_RatingValue1        0
commodity_RatingValue2        0
commodity_RatingValue3        0
commodity_RatingValue4        0
commodity_RatingValue5        0
commodity_Value_score       214
commodity_Quality_score     207
commodity_Taste_score       209
commodity_total_reviews       0
commodity_highlights         60
commodity_descriptio          0
dtype: int64

In [8]:
df.sort_values(by=['commodity_name'])  #會有相同的commodity_name	，但某些欄位不一樣

,commodity_name,commodity_images,commodity_price,commodity_url,commodity_description,commodity_average_rating,commodity_RatingValue1,commodity_RatingValue2,commodity_RatingValue3,commodity_RatingValue4,commodity_RatingValue5,commodity_Value_score,commodity_Quality_score,commodity_Taste_score,commodity_total_reviews,commodity_highlights,commodity_descriptio
492,1% Milkfat Small Curd Cottage Cheese - 16oz - ...,https://target.scene7.com/is/image/Target/GUES...,$1.99,https://www.target.com/p/1-milkfat-small-curd-...,Give your morning a flavorful start or power t...,3.00,3,0,0,0,3,3.50,2.33,3.00,6,"[Small-curd cottage cheese with 1% milk fat, M...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
227,1% Milkfat Small Curd Cottage Cheese - 24oz - ...,https://target.scene7.com/is/image/Target/GUES...,$2.99,https://www.target.com/p/1-milkfat-small-curd-...,Whether you're looking for a quick breakfast o...,2.96,11,2,2,1,11,2.92,2.38,2.38,27,"[Contains 24oz of low-fat cottage cheese, Made...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
286,4% Milkfat Small Curd Cottage Cheese - 24oz - ...,https://target.scene7.com/is/image/Target/GUES...,$2.99,https://www.target.com/p/4-milkfat-small-curd-...,With a container of 4% Milk Fat Small Curd Cot...,3.39,8,4,0,1,8,3.33,3.21,3.26,28,"[Small-curd cottage cheese with 4% milk fat, M...","[Contains: Milk, May Contain: Soy, Tree Nuts, ..."
803,8 Myles Frozen BBQ Mac and Cheese - 8oz,https://target.scene7.com/is/image/Target/GUES...,$4.49,https://www.target.com/p/8-myles-frozen-bbq-ma...,Whole grain elbow pasta tossed in a creamy whi...,5.00,0,0,0,0,0,5.00,5.00,5.00,3,"[Smokey BBQ Sauce, All real ingredients, Whole...","[Contains: Eggs, Milk, Wheat, Features: No Art..."
763,8 Myles Frozen BBQ Mac and Cheese - 8oz,https://target.scene7.com/is/image/Target/GUES...,$4.49,https://www.target.com/p/8-myles-frozen-bbq-ma...,Whole grain elbow pasta tossed in a creamy whi...,5.00,0,0,0,0,0,5.00,5.00,5.00,3,"[Smokey BBQ Sauce, All real ingredients, Whole...","[Contains: Eggs, Milk, Wheat, Features: No Art..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,Whisps Parmesan &#38; Asiago and Pepper Jack C...,https://target.scene7.com/is/image/Target/GUES...,$7.49,https://www.target.com/p/whisps-parmesan-38-as...,NaN,5.00,0,0,0,0,0,NaN,NaN,NaN,1,NaN,"[Contains: Milk, Form: Pieces, State of Readin..."
85,Whisps Parmesan Cheese Crisps - 2.12oz,https://target.scene7.com/is/image/Target/GUES...,$3.99,https://www.target.com/p/whisps-parmesan-chees...,Whisps are made from just one delicious ingred...,4.17,9,2,8,7,9,3.38,4.35,4.00,78,[100% Parmesan Cheese - the only artisanal cop...,"[Contains: Milk, Dietary Needs: Gluten Free, F..."
176,Whisps Tangy Ranch Cheese Crisps - 2.12oz,https://target.scene7.com/is/image/Target/GUES...,$3.69,https://www.target.com/p/whisps-tangy-ranch-ch...,A cheddar-for-you Ranch! What happens when we ...,3.55,2,2,0,2,2,2.67,5.00,3.00,11,"[10g of Protein per Serving, 3g Carbs per Serv...","[Contains: Milk, Form: Pieces, State of Readin..."
686,White Cheddar Mac and Cheese - 20oz - Good & G...,https://target.scene7.com/is/image/Target/GUES...,$3.99,https://www.target.com/p/white-cheddar-mac-and...,Take mac and cheese night up a notch with Whit...,4.68,1,1,0,3,1,4.70,4.85,4.57,31,[Mac and cheese made with whole milk and butte...,"[Contains: Eggs, Milk, Wheat, State of Readine..."
